## Coefficient Reloadable FIR Filter Hardware Design
This notebook demonstrates how to accelerate a function (FIR filter) from a processing system (PS) to programmable logic (PL), and how to reload FIR filter coefficients on a Zynq-7000 using PYNQ. First, input the voice signal. Then, realize the noise reduction of the voice signal in the PL, and then play the voice after noise reduction

## 查看音频参数

In [2]:
from IPython.display import Audio as IPAudio
IPAudio("/home/xilinx/jupyter_notebooks/entire_FIR/recording_0.wav")  #recording_0  test_recording

In [26]:
%matplotlib inline
import wave
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.fftpack import fft

wav_path = "/home/xilinx/jupyter_notebooks/entire_FIR/recording_0.wav"

with wave.open(wav_path, 'r') as wav_file:
    raw_frames = wav_file.readframes(-1)
    num_frames = wav_file.getnframes()
    num_channels = wav_file.getnchannels()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()



temp_buffer = np.empty((num_frames, num_channels, 4), dtype=np.uint8) #int8
raw_bytes = np.frombuffer(raw_frames, dtype=np.uint8)
temp_buffer[:, :, :sample_width] = raw_bytes.reshape(-1, num_channels, sample_width)
temp_buffer[:, :, sample_width:] = (temp_buffer[:, :, sample_width-1:sample_width] >> 7) * 255
frames = temp_buffer.view('<i4').reshape(temp_buffer.shape[:-1])
print(frames,temp_buffer)
print(num_frames,num_channels,sample_rate,sample_width,raw_bytes)    

[[-18137 -17228]
 [-16288 -17228]
 [-18820 -15814]
 ..., 
 [ 32929  30201]
 [ 32585  30747]
 [ 29615  29449]] [[[ 39 185 255 255]
  [180 188 255 255]]

 [[ 96 192 255 255]
  [180 188 255 255]]

 [[124 182 255 255]
  [ 58 194 255 255]]

 ..., 
 [[161 128   0   0]
  [249 117   0   0]]

 [[ 73 127   0   0]
  [ 27 120   0   0]]

 [[175 115   0   0]
  [  9 115   0   0]]]
240000 2 48000 3 [ 39 185 255 ...,   9 115   0]


### Program FPGA

In [27]:
from pynq import Overlay

# Program PL + load drivers for hardware found in HWH
overlay = Overlay("/home/xilinx/jupyter_notebooks/entire_FIR/fir_reload.bit")

### Load Drivers

In [28]:
from pynq.lib import AxiGPIO
from pynq import allocate

# Load FIR Data DMA
dma_fir_data = overlay.axi_dma_fir_data
# Load FIR Reload DMA
dma_fir_reload = overlay.axi_dma_reload
# Load FIR Config DMA
dma_fir_config = overlay.axi_dma_config

### paramater value

In [29]:
import numpy as np
import time
n=num_frames
T = n/num_frames
t = np.linspace(0,T,n,endpoint=False)
n_plot = 100

### Send Waveform to FIR Filter via DMA with initial coefficients

In [30]:
# Allocate physically contiguous buffers for direct register DMA
in_data_buffer = allocate(shape=(n,2),dtype=np.int32) #int32
print(in_data_buffer)
out_data_buffer = allocate(shape=(n,2),dtype=np.int32)

# Copy the samples to the input buffer
for channel_index in range(num_channels):
    samples = frames[:, channel_index]
    samples = samples.astype(np.int32)
    print(samples)
    np.copyto(in_data_buffer[:, channel_index],samples)
    
#samples = samples.astype(np.int32)
#np.copyto(in_data_buffer,samples)
# Trigger the DMA and wait for the result
start_time = time.time()
dma_fir_data.sendchannel.transfer(in_data_buffer)
dma_fir_data.recvchannel.transfer(out_data_buffer)
dma_fir_data.sendchannel.wait()
dma_fir_data.recvchannel.wait()
stop_time = time.time()
hw_exec_time = stop_time-start_time
print('Hardware FIR execution time: ',hw_exec_time,'s for ',T,'s signal')
#print('Hardware acceleration factor: ',sw_exec_time_lpf / hw_exec_time)

print(out_data_buffer[:,])
out_data_buffer.close()



[[0 0]
 [0 0]
 [0 0]
 ..., 
 [0 0]
 [0 0]
 [0 0]]
[-18137 -16288 -18820 ...,  32929  32585  29615]
[-17228 -17228 -15814 ...,  30201  30747  29449]
Hardware FIR execution time:  0.00645756721496582 s for  1.0 s signal
[[ 42  61]
 [ 58  37]
 [  1 -60]
 ..., 
 [154 264]
 [268 189]
 [244 191]]


In [31]:
from scipy.io.wavfile import write
write("还原.wav", sample_rate, out_data_buffer.astype(np.int16))

In [32]:
from IPython.display import Audio as IPAudio
IPAudio("/home/xilinx/jupyter_notebooks/entire_FIR/还原.wav")

In [17]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")
pAudio = base.audio


In [ ]:
pAudio.load("/home/xilinx/jupyter_notebooks/entire_FIR/recording_0.wav")
pAudio.play()